## Bad benchmarks

In [1]:
import time
from typing import Callable

In [2]:
def pass_stmt() -> Callable:
    for i in range(10_000_000):
        pass

In [3]:
def ellipses() -> None:
    for i in range(10_000_000):
        ...    

In [5]:
def get_time(func: Callable) -> None:
    start_time: float = time.perf_counter()
    func()
    end_time: float = time.perf_counter()
    print(f'"{func.__name__}()" took {end_time - start_time:.5f} seconds') 

In [6]:
get_time(ellipses)

"ellipses()" took 0.09288 seconds


In [8]:
get_time(pass_stmt)  # warmup
get_time(ellipses)
get_time(pass_stmt)

"pass_stmt()" took 0.09955 seconds
"ellipses()" took 0.06252 seconds
"pass_stmt()" took 0.06275 seconds


In [9]:
get_time(pass_stmt)  # warmup
get_time(ellipses)
get_time(pass_stmt)

"pass_stmt()" took 0.06125 seconds
"ellipses()" took 0.06359 seconds
"pass_stmt()" took 0.06264 seconds


timeit gives you better tools for timing your code

## Premature optimization

In [11]:
def calculate_sum(n):
    result = 0
    for i in range(n):
        result += 1
    return result

def calculate_sum_optimized(n):
    return (n * (n - 1)) // 2

def get_time(func: Callable, *args: int) -> None:
    start_time: float = time.perf_counter()
    func(*args)
    end_time: float = time.perf_counter()
    print(f'"{func.__name__}()" took {end_time - start_time:.5f} seconds')

In [15]:
n = 100_000_000
get_time(calculate_sum, n)
get_time(calculate_sum_optimized, n)

"calculate_sum()" took 1.40911 seconds
"calculate_sum_optimized()" took 0.00000 seconds


## Type annotations

Help avoid mistakes.  When you are tired.

Might give you context actions in code editor/IDE.

In [16]:
name: str = 'Bob'
age: int = '88'

# Noob mistakes

## Equality, identity

In [24]:
var_a: str = 'bob'
var_b: str = 'bob'

print(f'{id(var_a)=}')
print(f'{id(var_b)=}')
print(var_a == var_b)
print(var_a is var_b)

id(var_a)=4460179120
id(var_b)=4460179120
True
True


In [26]:
var_b: str = 'bob'.lower()
print(f'{id(var_a)=}')
print(f'{id(var_b)=}')
print(var_a == var_b)
print(var_a is var_b)

id(var_a)=4460179120
id(var_b)=4460202816
True
False


In [27]:
var_a: int = 100
var_b: int = 100

print(f'{id(var_a)=}')
print(f'{id(var_b)=}')
print(var_a == var_b)
print(var_a is var_b)

id(var_a)=4345329096
id(var_b)=4345329096
True
True


## Modifying list while iterating through it

In [30]:
items: list[str] = "A B C D E".split()
items

['A', 'B', 'C', 'D', 'E']

In [35]:
# supposedly won't work (but it does!)
for item in items:
    if item == 'B':
        items.remove('B')
    else:
        print(item)

A
C
D
E


In [39]:
# better, create new list
items: list[str] = "A B C D E".split()
new_items = []

for item in items:
    if item == 'B':
        continue
    else:
        new_items.append(item)

print(new_items)

['A', 'C', 'D', 'E']


## Modifying strings
and other immutable objects

In [40]:
from timeit import timeit

In [50]:
def append_text() -> str:
    text: str = ''
    for i in range(50):
        text += 'text'

    return text

def append_text_join() -> str:
    elements: list[str] = []
    for i in range(50):
        elements.append('text')

    return ''.join(elements)

print(f'Same result? {append_text() == append_text_join()=}')
warmup: float = timeit(append_text)
normal_time: float = timeit(append_text)
join_time: float = timeit(append_text_join)

Same result? append_text() == append_text_join()=True


In [53]:
print(f'normal: {normal_time:.5f}s')
print(f'join: {join_time:.5f}s')

normal: 0.68547s
join: 0.57648s


## Overcomplicating file handling

In [57]:
from typing import TextIO

file: TextIO = open('notes.txt')
content: str = file.read()
print(content)
file.close()  # hope this actually gets executed

Bob eats serial for breakfast.




In [58]:
# better
from typing import TextIO

file: TextIO = open('notes.txt')
try:
    content: str = file.read()
    print(content)
finally:
    file.close()  # close file regardless of what happens

Bob eats serial for breakfast.




In [60]:
# even better
from typing import TextIO

with open('notes.txt') as file:
    content: str = file.read()
    
print(content)


Bob eats serial for breakfast.




In [68]:
# even better still
from typing import TextIO
import logging

logging.basicConfig(level=logging.ERROR)
try:
    with open('nodtes.txt') as file:
        content: str = file.read()
except FileNotFoundError as e: # 
    logging.error(e)

print(content)


ERROR:root:[Errno 2] No such file or directory: 'nodtes.txt'


Bob eats serial for breakfast.




## Bad type checking

In [73]:
name: str = 'Bob'
number: int = 123

print(f'{type(name) == type(number)=}')
x: str = 'x'
print(f'{type(name) == type(x)=}')  # too strict for comparing types, inheritance

type(name) == type(number)=False
type(name) == type(x)=True


In [75]:
print(f'{isinstance(name, str)=}')
print(f'{isinstance(number, str)=}')

isinstance(name, str)=True
isinstance(number, str)=False


In [77]:
print(f'{isinstance(name, (str, tuple))=}')
print(f'{isinstance(number, (str, int))=}')

isinstance(name, (str, tuple))=True
isinstance(number, (str, int))=True


In [89]:
class Animal:
    ...

class Cat(Animal):
    ...


print(f'{isinstance(Cat(), Animal)=}')
print(f'{isinstance(Animal(), Cat)=}')
print(f'{type(Animal())=}')

isinstance(Cat(), Animal)=True
isinstance(Animal(), Cat)=False
type(Animal())=<class '__main__.Animal'>


## Enumeration

In [90]:
letters: str = 'ABCDE'
for i, letter in enumerate(letters):
    print(f'{i + 1}: {letter}')

1: A
2: B
3: C
4: D
5: E


In [91]:
# Better
for i, letter in enumerate(letters, 1):
    print(f'{i}: {letter}')

1: A
2: B
3: C
4: D
5: E


In [92]:
# or use keyword for legibility
for i, letter in enumerate(letters, start=1):
    print(f'{i}: {letter}')

1: A
2: B
3: C
4: D
5: E


## Referencing

In [99]:
a: list[int] = [1,2,3]
b: list[int] = a  # doesn't copy, just creates a reference/alias to a

In [100]:
print(f'{id(a)=}')
print(f'{id(b)=}')

id(a)=4508853120
id(b)=4508853120


In [101]:
a.append(111)
b.append(222)
print(f'{a=}')
print(f'{b=}')

a=[1, 2, 3, 111, 222]
b=[1, 2, 3, 111, 222]


## Shallow copy

In [103]:
from typing import Any
from copy import deepcopy

a: list[Any] = [1, ['A', 'B'], 3]

In [116]:
b: list[Any] = a.copy()  # copies everything except reference types
b.append(10)
print(f'{a=}')
print(f'{b=}')

a=[1, ['A', 'B'], 3]
b=[1, ['A', 'B'], 3, 10]


In [108]:
b[1][0] = 'CHANGED'

In [109]:
print(f'{a=}')
print(f'{b=}')

a=[1, ['CHANGED', 'B'], 3]
b=[1, ['CHANGED', 'B'], 3, 10]


In [110]:
b[0] = 999
print(f'{a=}')
print(f'{b=}')

a=[1, ['CHANGED', 'B'], 3]
b=[999, ['CHANGED', 'B'], 3, 10]


In [115]:
a: list[Any] = [1, ['A', 'B'], 3]
b: list[Any] = deepcopy(a)
b.append(10)
print(f'{a=}')
print(f'{b=}')
b[1][0] = 'CHANGED'
print(f'{a=}')
print(f'{b=}')

a=[1, ['A', 'B'], 3]
b=[1, ['A', 'B'], 3, 10]
a=[1, ['A', 'B'], 3]
b=[1, ['CHANGED', 'B'], 3, 10]


# Uses for underscores

## Semi-private variables

Underscore is a naming convention indicating how the variable is expected to be used.

In [121]:
from uuid import uuid4, UUID

class Lamp:
    def __init__(self, brand:str) -> None:
        self.brand = brand
        self._id: UUID = uuid4()

    def get_id(self) -> UUID:
        return self._id

# subclass
class SubLamp(Lamp):
    def __init__(self, brand: str) -> None:
        super().__init__(brand)

    def sublamp_method(self) -> None:
        print(self._id)


In [124]:
print(Lamp('bob')._id)  # No!
print(Lamp('ted').get_id())

81e591a4-9b3c-41b6-9d75-509e77215980
74c4c435-1839-4e30-953e-ed8e31c2efd4


## Name mangling

In [151]:
from uuid import uuid4, UUID

class Lamp:
    def __init__(self, brand:str) -> None:
        self.brand = brand
        self.__hidden_id: UUID = uuid4()

    # def get_id(self) -> UUID:
    #     return self._id

pam: Lamp = Lamp(brand="pam")
print(f'{pam.brand=}')
# print(f'{pam.__hidden_id=}')  
print(f'{pam._Lamp__hidden_id=}')  

pam.brand='pam'
pam._Lamp__hidden_id=UUID('57fcd479-493f-4523-8951-154af37a467f')


## Dunder methods

Let you override built-in functionality for types

In [156]:
from typing import Self

class CustomNumber:
    def __init__(self, value: int) -> None:
        self.value = value

n1: CustomNumber = CustomNumber(10)
n2: CustomNumber = CustomNumber(100)

print(f'{n1=}')
# print(f'{n1 + n2=}')  # error

n1=<__main__.CustomNumber object at 0x107452a50>


In [167]:
from typing import Self

class CustomNumber:
    def __init__(self, value: int) -> None:
        self.value = value

    def __add__(self, other: Self) -> int:
        return (self.value + other.value)

    def __repr__(self) -> str:
        return f'CustomNumber(value={self.value})'

    def __or__(self, other: Self) -> str:
        return f'{self.value} | {other.value}'

n1: CustomNumber = CustomNumber(10)
n2: CustomNumber = CustomNumber(100)

print(f'{n1=}')
print(f'{n1 + n2=}')
print(f'{n1 | n2}')


n1=CustomNumber(value=10)
n1 + n2=110
10 | 100


## Use system-reserved names

## Case

In [172]:
gangsta_weather: str = "thunder"

match gangsta_weather:
    case 'rain':
        print('Umbrella')
    case 'sunny':
        print('sunscreen')
    case _:
        print('huh?')

huh?
